In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time
import numpy as np
import biofeatures

## Config

In [ ]:
# riot_ip = '192.168.0.102'
# riot_ip = 'localhost'
riot_ip = '192.168.1.5'
riot_port = 31000
client = SimpleUDPClient(riot_ip, riot_port) 

In [ ]:
def process_riot_data(unused_addr, args, values):
    global resp_data
    global t0
    
    # Get features address
    B = args[0]

#    # String
#     new_data = values
#     resp_data.append((new_data))

    # Float array
    new_data = values.split("\t")[5]
    resp_data.append(float(new_data))

    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    else: 
#         print(new_data)
        pass
        

    warmup_t = 10
    tf = time.time()

    if tf-t0 > warmup_t: 
        if not B.is_warmed_up:
            print("WARMUP")
            B.set_data(resp_data[-B.buffer_length:])
            B.is_warmed_up = True
            B.update_loop()

    # only save the last 5 min of data
    if len(resp_data) > 200 * 60 * 5:
        resp_data = resp_data[-200*60*5:]

In [ ]:
B = biofeatures.breathing(data = np.ones(10), buffer_length=2000)
resp_data = []
t0 = time.time()
ti = time.time()
riot_dispatcher = dispatcher.Dispatcher()
riot_dispatcher.map("/raw/", process_riot_data, B)

server = osc_server.ThreadingOSCUDPServer((riot_ip, riot_port), riot_dispatcher)
print(time.strftime("%H:%M:%S", time.gmtime()))
print("Serving on {}".format(server.server_address))
try:
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
except:
    raise

In [ ]:
B

In [ ]:
B.features

In [ ]:
## RESUME
# print(time.strftime("%H:%M:%S", time.gmtime()))
# print("Serving on {}".format(server.server_address))
# try:
#     B.update_data_flag = True
#     B.update_loop()
#     server.serve_forever()
# except KeyboardInterrupt:
#     B.update_data_flag = False
# except:
#     raise